<a href="https://colab.research.google.com/github/Arwah45/DepthEstimation/blob/main/DepthEstimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch torchvision transformers diffusers datasets tqdm matplotlib Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is i

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from datasets import load_dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F
from transformers import DPTFeatureExtractor, DPTForDepthEstimation

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
 #Define transformations
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

In [ ]:
class BokehDataset(Dataset):
    def __init__(self, ds, split='train', transform=None):
        self.ds = ds[split]
        self.transform = transform

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]

        # Access the original and bokeh images from the item dictionary
        original_img = item['original_image']  # Already a PIL image
        bokeh_img = item['bokeh_image']  # Already a PIL image

        # Apply transformations if provided
        if self.transform:
            original_img = self.transform(original_img)
            bokeh_img = self.transform(bokeh_img)

        return original_img, bokeh_img


In [ ]:
ds = load_dataset("comHannah/bokeh-dataset")
bokeh_dataset = BokehDataset(ds, split='train', transform=transform)
dataloader = DataLoader(bokeh_dataset, batch_size=4, shuffle=True, num_workers=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4400 [00:00<?, ? examples/s]

In [ ]:
print(ds['train'][0])

{'original_image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1550x1024 at 0x7E24276C4E80>, 'edit_prompt': 'Transform the image with a bokeh-inspired blur effect.', 'bokeh_image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1550x1024 at 0x7E22FBDC25F0>}


In [ ]:
print(bokeh_dataset[0])



(tensor([[[0.8314, 0.8353, 0.8392,  ..., 0.5725, 0.5725, 0.5608],
         [0.8353, 0.8392, 0.8431,  ..., 0.5725, 0.5765, 0.5765],
         [0.8392, 0.8392, 0.8431,  ..., 0.5725, 0.5804, 0.5765],
         ...,
         [0.9294, 0.9333, 0.9373,  ..., 0.9020, 0.9020, 0.9059],
         [0.9255, 0.9294, 0.9373,  ..., 0.9059, 0.9059, 0.9098],
         [0.9255, 0.9333, 0.9412,  ..., 0.9137, 0.9137, 0.9137]],

        [[0.7765, 0.7765, 0.7725,  ..., 0.3647, 0.3647, 0.3529],
         [0.7725, 0.7725, 0.7725,  ..., 0.3608, 0.3647, 0.3569],
         [0.7686, 0.7686, 0.7725,  ..., 0.3529, 0.3569, 0.3490],
         ...,
         [0.8824, 0.8824, 0.8824,  ..., 0.8549, 0.8549, 0.8549],
         [0.8863, 0.8824, 0.8824,  ..., 0.8510, 0.8510, 0.8471],
         [0.8863, 0.8824, 0.8824,  ..., 0.8549, 0.8549, 0.8510]],

        [[0.7255, 0.7255, 0.7176,  ..., 0.0863, 0.0863, 0.0706],
         [0.7216, 0.7216, 0.7216,  ..., 0.0980, 0.0980, 0.0941],
         [0.7216, 0.7216, 0.7216,  ..., 0.1020, 0.1059, 0

In [ ]:
class CustomControlNet(nn.Module):
    def __init__(self, input_channels=3, depth_channels=1, hidden_channels=64):
        super().__init__()
        self.input_conv = nn.Conv2d(input_channels + depth_channels, hidden_channels, kernel_size=3, padding=1)
        self.conv1 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, padding=1)
        self.output_conv = nn.Conv2d(hidden_channels, input_channels, kernel_size=3, padding=1)
        self.activation = nn.ReLU()

    def forward(self, x, depth):
    # Ensure depth map has an extra dimension for channels
        depth = depth.unsqueeze(1)  # Add channel dimension to depth map
        combined = torch.cat([x, depth], dim=1)  # Concatenate along the channel dimension
        out = self.activation(self.input_conv(combined))
        out = self.activation(self.conv1(out))
        out = self.activation(self.conv2(out))
        return torch.sigmoid(self.output_conv(out))


controlnet = CustomControlNet().to(device)

In [ ]:
def generate_depth_maps(images, device):
    feature_extractor = DPTFeatureExtractor.from_pretrained("Intel/dpt-large")
    model = DPTForDepthEstimation.from_pretrained("Intel/dpt-large").to(device)

    depth_maps = []
    with torch.no_grad():
        for img in images:
            # Convert tensor to PIL Image
            pil_img = transforms.ToPILImage()(img.cpu())
            inputs = feature_extractor(images=pil_img, return_tensors="pt").to(device)
            outputs = model(**inputs)
            predicted_depth = outputs.predicted_depth
            depth_map = (predicted_depth - predicted_depth.min()) / (predicted_depth.max() - predicted_depth.min())
            # Resize depth map to match input image size
            depth_map = F.interpolate(depth_map.unsqueeze(1), size=img.shape[-2:], mode='bilinear', align_corners=False)
            depth_maps.append(depth_map.squeeze())

    return torch.stack(depth_maps).to(device)

In [ ]:
def train_controlnet(model, dataloader, num_epochs, device):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            original_imgs, bokeh_imgs = batch
            original_imgs = original_imgs.to(device)
            bokeh_imgs = bokeh_imgs.to(device)

            depth_maps = generate_depth_maps(original_imgs, device)

            optimizer.zero_grad()
            outputs = model(original_imgs, depth_maps)
            loss = criterion(outputs, bokeh_imgs)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader):.4f}")

    return model

In [ ]:
def apply_bokeh_effect(model, image, device):
    model.eval()
    with torch.no_grad():
        depth_map = generate_depth_maps(image.unsqueeze(0), device)
        bokeh_image = model(image.unsqueeze(0), depth_map)
    return bokeh_image.squeeze(0)

In [ ]:
def psnr(img1, img2):
    mse = F.mse_loss(img1, img2)
    return 20 * torch.log10(1.0 / torch.sqrt(mse))

In [ ]:
def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2
    img1 = img1.unsqueeze(0)
    img2 = img2.unsqueeze(0)
    mu1 = F.avg_pool2d(img1, 11, stride=1, padding=5)
    mu2 = F.avg_pool2d(img2, 11, stride=1, padding=5)
    sigma1_sq = F.avg_pool2d(img1**2, 11, stride=1, padding=5) - mu1**2
    sigma2_sq = F.avg_pool2d(img2**2, 11, stride=1, padding=5) - mu2**2
    sigma12 = F.avg_pool2d(img1 * img2, 11, stride=1, padding=5) - mu1 * mu2
    ssim_map = ((2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)) / ((mu1**2 + mu2**2 + C1) * (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

In [ ]:
def assess_quality(img1, img2):
    psnr_value = psnr(img1, img2)
    ssim_value = ssim(img1, img2)
    return psnr_value.item(), ssim_value.item()

In [ ]:
def visualize_results(original_image, depth_map, bokeh_image):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    axs[0].imshow(original_image.permute(1, 2, 0).cpu().numpy())
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    axs[1].imshow(depth_map.squeeze().cpu().numpy(), cmap='viridis')
    axs[1].set_title('Depth Map')
    axs[1].axis('off')

    axs[2].imshow(bokeh_image.permute(1, 2, 0).cpu().numpy())
    axs[2].set_title('Bokeh Effect')
    axs[2].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
if __name__ == "__main__":
    # Train the ControlNet
    num_epochs = 10  # Adjust as needed
    trained_controlnet = train_controlnet(controlnet, dataloader, num_epochs, device)

    # Test the model
    sample_image, _ = next(iter(dataloader))
    sample_image = sample_image[0].to(device)

    bokeh_image = apply_bokeh_effect(trained_controlnet, sample_image, device)

    # Generate depth map for visualization
    depth_map = generate_depth_maps(sample_image.unsqueeze(0), device)

    # Visualize results
    visualize_results(sample_image, depth_map.squeeze(0), bokeh_image)

    # Assess quality
    psnr_value, ssim_value = assess_quality(sample_image, bokeh_image)
    print(f"PSNR: {psnr_value:.2f}, SSIM: {ssim_value:.2f}")

Epoch 1/10:   0%|          | 0/1100 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


preprocessor_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/dpt/feature_extraction_dpt.py:28: FutureWarning: The class DPTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DPTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/942 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/10:   0%|          | 1/1100 [01:55<35:09:22, 115.16s/it]Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a